In [6]:
from ariautils.midi import MidiDict
import torch
from transformers import AutoTokenizer
from src.model.model import MidiAria

tokenizer = AutoTokenizer.from_pretrained(
    "loubb/aria-medium-base",
    trust_remote_code=True,
    add_eos_token=True,
    add_dim_token=False
)
# style
checkpoint_path = "../checkpoints/aria/aria-style-epoch=06-val_loss=2.0712.ckpt"

# harmonization
# checkpoint_path = "../checkpoints/aria/aria-harmony-epoch=02-val_loss=4.2002.ckpt"

# old
# checkpoint_path = "../checkpoints/aria/aria-harmony-epoch=03-val_loss=4.4451.ckpt"

model = MidiAria(tokenizer, None)
model.to_lora()
state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))['state_dict']
model.load_state_dict(state_dict)


device = "mps" if torch.backends.mps.is_available() else "cpu"
torch.Tensor.cuda = lambda self, *args, **kwargs: self.to(device)


PROMPT_MIDI_LOAD_PATH = "../data/test/melody_short.mid"
CONTINUATION_MIDI_SAVE_PATH = "../data/test/continuation.mid"
midi_dict = MidiDict.from_midi(PROMPT_MIDI_LOAD_PATH)
tokens = tokenizer.tokenize(midi_dict, add_eos_token=True, add_dim_token=True)
token_ids = tokenizer._tokenizer.encode(tokens)
prompt_input_ids = torch.tensor([token_ids], device='cpu')

continuation = model.model.generate(
    prompt_input_ids.to('cpu'),
    max_length=1024,
    do_sample=True,
    temperature=0.97,
    top_p=0.95,
    use_cache=True,
)


# prompt_len = prompt_input_ids.shape[1]
prompt_len = 0
generated_tokens = continuation[0][prompt_len:]  # only the new part

# decode just the continuation
midi_dict = tokenizer.decode(generated_tokens.tolist())
print(tokenizer._tokenizer.decode(generated_tokens.tolist()))
midi_dict.to_midi().save(CONTINUATION_MIDI_SAVE_PATH)

print(f"Saved continuation to {CONTINUATION_MIDI_SAVE_PATH}")


[('prefix', 'instrument', 'piano'), '<S>', ('piano', 70, 80), ('onset', 0), ('dur', 470), ('piano', 74, 80), ('onset', 500), ('dur', 240), ('piano', 75, 80), ('onset', 750), ('dur', 1180), ('piano', 70, 80), ('onset', 2000), ('dur', 470), ('piano', 74, 80), ('onset', 2500), ('dur', 240), ('piano', 75, 80), ('onset', 2750), ('dur', 1180), ('piano', 70, 80), ('onset', 4000), ('dur', 470), ('piano', 74, 80), ('onset', 4500), ('dur', 240), ('piano', 75, 80), ('onset', 4750), ('dur', 710), '<T>', ('piano', 79, 80), ('onset', 500), ('dur', 240), ('piano', 80, 80), ('onset', 750), ('dur', 710), ('piano', 79, 80), ('onset', 1500), ('dur', 950), ('piano', 75, 80), ('onset', 2500), ('dur', 240), ('piano', 72, 80), ('onset', 2750), ('dur', 240), ('piano', 70, 80), ('onset', 3000), ('dur', 950), ('piano', 77, 80), ('onset', 4500), ('dur', 240), ('piano', 75, 80), ('onset', 4750), ('dur', 1180), '<T>', ('piano', 67, 80), ('onset', 1500), ('dur', 240), ('piano', 68, 80), ('onset', 1750), ('dur', 118